In [ ]:
#Manipulacao de CSV Zendesk - Candidata Yasmin Couto

In [ ]:
#Importando bibliotecas e abrindo arquivos csv
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
files.upload()

Saving pedidos.csv to pedidos.csv
Saving tickets.csv to tickets.csv
Saving chats.csv to chats.csv


In [26]:
#1 -Carregando arquivos e criando dataframes

#Criando dataframe df_chats
df_chats = pd.read_csv('chats.csv')  
df_chats ['ticket_id']= df_chats ['ticket_id'].astype(str)
df_chats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   chat_id                200000 non-null  object
 1   ticket_id              200000 non-null  object
 2   ChatStartDateBRT       200000 non-null  object
 3   chat_service_duration  200000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 6.1+ MB


In [27]:
#Criando dataframe df_pedidos

df_pedidos = pd.read_csv('pedidos.csv') 
df_pedidos ['order_id']= df_pedidos ['order_id'].astype(str)
df_pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108238 entries, 0 to 108237
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   order_id             108238 non-null  object
 1   OrderDateBRT         108238 non-null  object
 2   product_type         108236 non-null  object
 3   payment_type         108238 non-null  object
 4   cancellation_policy  86628 non-null   object
dtypes: object(5)
memory usage: 4.1+ MB


In [28]:
#Criando dataframe df_tickets

df_tickets = pd.read_csv('tickets.csv') 
df_tickets ['ticket_id']= df_tickets ['ticket_id'].astype(str)
df_tickets.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ticket_id             200000 non-null  object 
 1   ticket_type           163226 non-null  object 
 2   ticket_type_2         101791 non-null  object 
 3   ticket_type_3         85269 non-null   object 
 4   TicketCreatedDateBRT  200000 non-null  object 
 5   TicketSolvedDateBRT   198235 non-null  object 
 6   original_ticket_id    55846 non-null   float64
 7   order_id              152481 non-null  object 
dtypes: float64(1), object(7)
memory usage: 12.2+ MB


In [29]:
#Unindo camadas em um unico dataframe df

temp1 = pd.merge(df_tickets, df_chats,how='left', left_on='ticket_id', right_on='ticket_id')
df = pd.merge(temp1, df_pedidos,how='left', left_on='order_id', right_on='order_id')
df['TicketSolvedDateBRT'] = df['TicketSolvedDateBRT'].astype('datetime64[ns]')
df['TicketCreatedDateBRT'] = df['TicketCreatedDateBRT'].astype('datetime64[ns]')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   ticket_id              200000 non-null  object        
 1   ticket_type            163226 non-null  object        
 2   ticket_type_2          101791 non-null  object        
 3   ticket_type_3          85269 non-null   object        
 4   TicketCreatedDateBRT   200000 non-null  datetime64[ns]
 5   TicketSolvedDateBRT    198235 non-null  datetime64[ns]
 6   original_ticket_id     55846 non-null   float64       
 7   order_id               152481 non-null  object        
 8   chat_id                47846 non-null   object        
 9   ChatStartDateBRT       47846 non-null   object        
 10  chat_service_duration  47846 non-null   float64       
 11  OrderDateBRT           85031 non-null   object        
 12  product_type           85030 non-null   obje

In [30]:
#2 - Criando indicadores para os chamados

In [31]:
#Criando indicador - Resolvido (S/N)

def resolucao(id):
    if id is pd.NaT:
        return 'N'
    else:
        return 'S'
    return ''

df['Resolvido?'] = df['TicketSolvedDateBRT'].apply(resolucao)

print('Tickets resolvidos: ',len(df.loc[df['Resolvido?']=='S']),"-",round((len( df.loc[df['Resolvido?']=='S'])/len(df)*100)),'% do Total')
print('Tickets não resolvidos: ',len( df.loc[df['Resolvido?']=='N']),"-",round((len( df.loc[df['Resolvido?']=='N'])/len(df)*100)),'% do Total')

df.head(1)

Tickets resolvidos:  198235 - 99 % do Total
Tickets não resolvidos:  1765 - 1 % do Total


,ticket_id,ticket_type,ticket_type_2,ticket_type_3,TicketCreatedDateBRT,TicketSolvedDateBRT,original_ticket_id,order_id,chat_id,ChatStartDateBRT,chat_service_duration,OrderDateBRT,product_type,payment_type,cancellation_policy,Resolvido?
0,5362222,Solicitação de Cancelamento,Estorno,Alteração Na Quantidade De Hóspedes,2020-11-18 21:04:59,2021-05-28 15:06:01,NaN,5216867,NaN,NaN,NaN,2019-12-12,pacote,Cartão de crédito,ATÉ 2 SEMANAS DA COMPRAo cancelamento é GRATUI...,S


In [32]:
#Criando indicador - Tempo de Solução (dias)

df['TicketSolvedDateBRT'] = df['TicketSolvedDateBRT'].astype('datetime64[ns]')
df['TicketCreatedDateBRT'] = df['TicketCreatedDateBRT'].astype('datetime64[ns]')

df['Dia de Criação'] = (df['TicketCreatedDateBRT']).dt.weekday
df['Tempo de Solução']=(df['TicketSolvedDateBRT'] - df['TicketCreatedDateBRT']).dt.days
df.head(1)

,ticket_id,ticket_type,ticket_type_2,ticket_type_3,TicketCreatedDateBRT,TicketSolvedDateBRT,original_ticket_id,order_id,chat_id,ChatStartDateBRT,chat_service_duration,OrderDateBRT,product_type,payment_type,cancellation_policy,Resolvido?,Dia de Criação,Tempo de Solução
0,5362222,Solicitação de Cancelamento,Estorno,Alteração Na Quantidade De Hóspedes,2020-11-18 21:04:59,2021-05-28 15:06:01,NaN,5216867,NaN,NaN,NaN,2019-12-12,pacote,Cartão de crédito,ATÉ 2 SEMANAS DA COMPRAo cancelamento é GRATUI...,S,2,190.0


In [33]:
#Criando indicador - Tempo em Aberto (dias)

#Se o ticket está resolvido o Tempo em Aberto = Tempo de Solução
#Se o ticket não está resolvido o Tempo em Aberto = hoje - Data de Criação


hoje = np.datetime64('today')

def tempo_aberto(x,y,z):
    if x == "N":
        return (hoje - y).days
    else:
        return z
    return ''

df['Tempo em Aberto'] = df.apply(lambda x: tempo_aberto(x['Resolvido?'], x['TicketCreatedDateBRT'],x['Tempo de Solução']), axis=1)
df.head(1)

,ticket_id,ticket_type,ticket_type_2,ticket_type_3,TicketCreatedDateBRT,TicketSolvedDateBRT,original_ticket_id,order_id,chat_id,ChatStartDateBRT,chat_service_duration,OrderDateBRT,product_type,payment_type,cancellation_policy,Resolvido?,Dia de Criação,Tempo de Solução,Tempo em Aberto
0,5362222,Solicitação de Cancelamento,Estorno,Alteração Na Quantidade De Hóspedes,2020-11-18 21:04:59,2021-05-28 15:06:01,NaN,5216867,NaN,NaN,NaN,2019-12-12,pacote,Cartão de crédito,ATÉ 2 SEMANAS DA COMPRAo cancelamento é GRATUI...,S,2,190.0,190.0


In [34]:
#Canais de contato

def canaldecontato(id):
    if id == 0:
        return 'email'
    else:
        return 'chat'
    return ''

df['chat_id'] = df['chat_id'].replace(np.nan, 0)
df['Canal de contato'] = df['chat_id'].apply(canaldecontato)


print('Tickets via email: ',len( df.loc[df['Canal de contato']=='email']),"-",round((len( df.loc[df['Canal de contato']=='email'])/len(df)*100)),'% do Total')
print('Tickets via chat: ',len(df.loc[df['Canal de contato']=='chat']),"-",round((len( df.loc[df['Canal de contato']=='chat'])/len(df)*100)),'% do Total')

df.head(1)

Tickets via email:  152154 - 76 % do Total
Tickets via chat:  47846 - 24 % do Total


,ticket_id,ticket_type,ticket_type_2,ticket_type_3,TicketCreatedDateBRT,TicketSolvedDateBRT,original_ticket_id,order_id,chat_id,ChatStartDateBRT,chat_service_duration,OrderDateBRT,product_type,payment_type,cancellation_policy,Resolvido?,Dia de Criação,Tempo de Solução,Tempo em Aberto,Canal de contato
0,5362222,Solicitação de Cancelamento,Estorno,Alteração Na Quantidade De Hóspedes,2020-11-18 21:04:59,2021-05-28 15:06:01,NaN,5216867,0,NaN,NaN,2019-12-12,pacote,Cartão de crédito,ATÉ 2 SEMANAS DA COMPRAo cancelamento é GRATUI...,S,2,190.0,190.0,email


In [35]:
#2 - Criando indicadores para as pedidos

In [36]:
#Trazendo informações dos tickets para a compra

df_pedidos = pd.merge(df_pedidos,df[['ticket_id','Resolvido?','Tempo em Aberto','order_id','TicketCreatedDateBRT']],how='left', left_on='order_id', right_on='order_id')
df_pedidos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139945 entries, 0 to 139944
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   order_id              139945 non-null  object        
 1   OrderDateBRT          139945 non-null  object        
 2   product_type          139943 non-null  object        
 3   payment_type          139945 non-null  object        
 4   cancellation_policy   113859 non-null  object        
 5   ticket_id             85031 non-null   object        
 6   Resolvido?            85031 non-null   object        
 7   Tempo em Aberto       85031 non-null   float64       
 8   TicketCreatedDateBRT  85031 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 10.7+ MB


In [37]:
#Checando quais compras abriram tickets
#Criando indicador - Ticket? (S/N)

def ticket_criado(id):
    if id == '-':
        return 'N'
    else:
        return 'S'
    return ''

df_pedidos['ticket_id'] = df_pedidos['ticket_id'].replace(np.nan, '-')
df_pedidos['Resolvido?'] = df_pedidos['Resolvido?'].replace(np.nan, '-')
df_pedidos['Tempo em Aberto'] = df_pedidos['Tempo em Aberto'].replace(np.nan, '-')
df_pedidos['TicketCreatedDateBRT'] = df_pedidos['TicketCreatedDateBRT'].replace(np.nan, '-')


df_pedidos['Ticket?'] = df_pedidos['ticket_id'].apply(ticket_criado)

print('Compras que abriram ticket: ',len(df_pedidos.loc[df_pedidos['Ticket?']=='S']),"-",round((len( df_pedidos.loc[df_pedidos['Ticket?']=='S'])/len(df_pedidos)*100)),'% do Total')
print('Compras que não abriram ticket: ',len( df_pedidos.loc[df_pedidos['Ticket?']=='N']),"-",round((len( df_pedidos.loc[df_pedidos['Ticket?']=='N'])/len(df_pedidos)*100)),'% do Total')

Compras que abriram ticket:  85031 - 61 % do Total
Compras que não abriram ticket:  54914 - 39 % do Total


In [38]:
#Organizando dataframe para exportação do arquivo csv

df_pedidos = df_pedidos.rename(columns={'order_id': 'ID Compra', 'OrderDateBRT': 'Data da Compra', 'product_type': 'Produto', 'payment_type': 'Forma de Pagamento','cancellation_policy': 'Política de Cancelamento', 'ticket_id': 'ID Ticket', 'product_type': 'Produto', 'payment_type': 'Forma de Pagamento','TicketCreatedDateBRT':'Data do Ticket'})

df_pedidos = df_pedidos[['ID Compra', 'Data da Compra', 'Produto', 'Forma de Pagamento',
       'Política de Cancelamento','Ticket?', 'Resolvido?','Tempo em Aberto', 'Data do Ticket','ID Ticket']]

df_pedidos.head(1)

,ID Compra,Data da Compra,Produto,Forma de Pagamento,Política de Cancelamento,Ticket?,Resolvido?,Tempo em Aberto,Data do Ticket,ID Ticket
0,6195334,2020-07-07,pacote,Boleto,ANTES DA CONFIRMAÇÃO DO VOOo cancelamento éGRA...,N,-,-,-,-


In [39]:
df_pedidos.to_csv('pedidos_final.csv')

In [40]:
#3 - Criando indicadores de volume

In [41]:
#Demanda de chamados por turno
import datetime
from datetime import datetime

def turno(i):
    x=i.hour
    if (x>=0) & (x<8):
        return 'Madrugada'
    elif (x>=8) & (x<12):
        return 'Manhã'
    elif (x>=12) & (x<18):
        return 'Tarde'
    elif (x>=18) & (x<=23):
        return 'Noite'

    
df['Turno'] = df['TicketCreatedDateBRT'].apply(turno)

print('Tickets abertos das 8h às 12h: ',len(df.loc[df['Turno']=='Manhã']),"-",round((len( df.loc[df['Turno']=='Manhã'])/len(df)*100)),'% do Total')
print('Tickets abertos das 12h às 18h: ',len(df.loc[df['Turno']=='Tarde']),"-",round((len( df.loc[df['Turno']=='Tarde'])/len(df)*100)),'% do Total')
print('Tickets abertos das 18h às 0h: ',len(df.loc[df['Turno']=='Noite']),"-",round((len( df.loc[df['Turno']=='Noite'])/len(df)*100)),'% do Total')
print('Tickets abertos das 0h às 8h: ',len(df.loc[df['Turno']=='Madrugada']),"-",round((len( df.loc[df['Turno']=='Madrugada'])/len(df)*100)),'% do Total')

df.head(1)

Tickets abertos das 8h às 12h:  47988 - 24 % do Total
Tickets abertos das 12h às 18h:  80140 - 40 % do Total
Tickets abertos das 18h às 0h:  53060 - 27 % do Total
Tickets abertos das 0h às 8h:  18812 - 9 % do Total


,ticket_id,ticket_type,ticket_type_2,ticket_type_3,TicketCreatedDateBRT,TicketSolvedDateBRT,original_ticket_id,order_id,chat_id,ChatStartDateBRT,...,OrderDateBRT,product_type,payment_type,cancellation_policy,Resolvido?,Dia de Criação,Tempo de Solução,Tempo em Aberto,Canal de contato,Turno
0,5362222,Solicitação de Cancelamento,Estorno,Alteração Na Quantidade De Hóspedes,2020-11-18 21:04:59,2021-05-28 15:06:01,NaN,5216867,0,NaN,...,2019-12-12,pacote,Cartão de crédito,ATÉ 2 SEMANAS DA COMPRAo cancelamento é GRATUI...,S,2,190.0,190.0,email,Noite


In [42]:
#Demanda de chamados por dia da semana

import calendar

def dia(i):
    x=calendar.day_name[i.weekday()]
    if x=="Monday":
        return 'Segunda-feira'
    elif x=="Tuesday":
        return 'Terça-feira'
    elif x=="Wednesday":
        return 'Quarta-feira'
    elif x=="Thursday":
        return 'Quinta-feira'
    elif x=="Friday":
        return 'Sexta-feira'
    elif x=="Saturday":
        return 'Sábado'
    elif x=="Sunday":
        return 'Domingo'


df['Dia'] = df['TicketCreatedDateBRT'].apply(dia)
df.head(1)

,ticket_id,ticket_type,ticket_type_2,ticket_type_3,TicketCreatedDateBRT,TicketSolvedDateBRT,original_ticket_id,order_id,chat_id,ChatStartDateBRT,...,product_type,payment_type,cancellation_policy,Resolvido?,Dia de Criação,Tempo de Solução,Tempo em Aberto,Canal de contato,Turno,Dia
0,5362222,Solicitação de Cancelamento,Estorno,Alteração Na Quantidade De Hóspedes,2020-11-18 21:04:59,2021-05-28 15:06:01,NaN,5216867,0,NaN,...,pacote,Cartão de crédito,ATÉ 2 SEMANAS DA COMPRAo cancelamento é GRATUI...,S,2,190.0,190.0,email,Noite,Quarta-feira


In [43]:
df.to_csv('tickets_final.csv')

In [44]:
#Número de atendimentos por motivos de contato

df_motivos = pd.DataFrame(df.groupby(['ticket_type','ticket_type_2','ticket_type_3','Canal de contato'])['ticket_id'].count().reset_index())
df_motivos['chave']=df['ticket_type']+df['ticket_type_2']+df['ticket_type_3']
df_motivos = df_motivos.rename(columns={'ticket_type':'Categoria 1','ticket_type_2':'Categoria 2','ticket_type_3':'Categoria 3','ticket_id':'No de Chamados'})

df_motivos = df_motivos.sort_values(by=['No de Chamados'],ascending=False)

df_motivos.head(1)

,Categoria 1,Categoria 2,Categoria 3,Canal de contato,No de Chamados,chave
95,Informações ou Dúvidas,Hotel Pacote,Regras De Cancelamento,email,6815,SolicitaçãoAlteração De Dados CadastraisAltera...


In [45]:
#Atendimentos com maior tempo médio de solução por motivo de contato

temp1 = df.loc[df['Resolvido?']=='S']
df_solucao = pd.DataFrame(temp1.groupby(['ticket_type','ticket_type_2','ticket_type_3','Canal de contato'])['Tempo de Solução'].mean().round().reset_index())
df_solucao['chave']=df['ticket_type']+df['ticket_type_2']+df['ticket_type_3']
df_solucao = df_solucao.rename(columns={'ticket_type':'Categoria 1','ticket_type_2':'Categoria 2','ticket_type_3':'Categoria 3'})

df_solucao = df_solucao.sort_values(by=['Tempo de Solução'],ascending=False)

df_solucao.head(1)

,Categoria 1,Categoria 2,Categoria 3,Canal de contato,Tempo de Solução,chave
416,Solicitação,Remarcação 2021,Falecimento,email,219.0,Informações ou DúvidasHotel PacoteTaxas E Juros


In [46]:
#Numero de chamados e tempo médio de solução por motivos de contato

df_motivo_solucao = pd.merge(df_motivos, df_solucao, left_on=('Canal de contato','Categoria 1', 'Categoria 2','Categoria 3', 'chave'), right_on=('Canal de contato','Categoria 1', 'Categoria 2','Categoria 3', 'chave'))
df_motivo_solucao=df_motivo_solucao[['No de Chamados', 'Tempo de Solução','Canal de contato','Categoria 1', 'Categoria 2', 'Categoria 3']]
df_motivo_solucao = df_motivo_solucao.sort_values(by=['No de Chamados'],ascending=False)

df_motivo_solucao.head(1)

,No de Chamados,Tempo de Solução,Canal de contato,Categoria 1,Categoria 2,Categoria 3
0,6815,7.0,email,Informações ou Dúvidas,Hotel Pacote,Regras De Cancelamento


In [47]:
#No de chamados por turno, dia e canal
df_dia_turno_tempo = pd.DataFrame(df.groupby(['Canal de contato','Dia','Turno'])['Tempo de Solução'].mean().round().reset_index())
df_dia_turno_tempo['chave']=df['Canal de contato']+df['Dia']+df['Turno']
df_dia_turno_tempo = df_dia_turno_tempo.rename(columns={'ticket_id':'No de chamados'})

df_dia_turno_tempo = df_dia_turno_tempo.sort_values(by=['Tempo de Solução'],ascending=True)

df_dia_turno_tempo.head(1)

,Canal de contato,Dia,Turno,Tempo de Solução,chave
22,chat,Sábado,Noite,11.0,emailSexta-feiraTarde


In [49]:
#Número de chamados por turno, dia e canal

df_dia_turno = pd.DataFrame(df.groupby(['Canal de contato','Dia','Turno'])['ticket_id'].count().reset_index())
df_dia_turno['chave']=df_dia_turno['Canal de contato']+df_dia_turno['Dia']+df_dia_turno['Turno']
df_dia_turno = df_dia_turno.rename(columns={'ticket_id':'No de chamados'})

df_dia_turno = df_dia_turno.sort_values(by=['No de chamados'],ascending=True)

df_dia_turno.head(1)


,Canal de contato,Dia,Turno,No de chamados,chave
0,chat,Domingo,Madrugada,384,chatDomingoMadrugada


In [62]:
#Numero de chamados e Tempo de Solução médio por dia, turno e canal de contato
df_volume_dia_turno = pd.merge(df_dia_turno_tempo, df_dia_turno,how='left', left_on=('chave'), right_on=('chave'))
df_volume_dia_turno=df_volume_dia_turno[['Canal de contato_x', 'Dia_x', 'Turno_x', 'Tempo de Solução','No de chamados']]
df_dia_turno_tempo = df_dia_turno_tempo.rename(columns={'Canal de contato_x':'Canal de contato', 'Dia_x':'Dia', 'Turno_x':'Turno'})

df_volume_dia_turno = df_volume_dia_turno.sort_values(by=['No de chamados'],ascending=False)

df_volume_dia_turno.head(1)

,Canal de contato_x,Dia_x,Turno_x,Tempo de Solução,No de chamados
0,chat,Sábado,Noite,11.0,9284


In [63]:
df_volume_dia_turno.to_csv('volume_final.csv')